## Variaciones sobre el modelo inicial

Vamos a implementar alguna modificación sobre el modelo inicial propuesto en la primera sesión.

Para empezar vamos a definir un conjunto de validación. Con este conjunto de calidación extraído del propio conjunto de entrenamiento estimaremos los mejores parámetros. Dejaremos el conjunto de test sólo para estimar el acierto final con estos mejores parámetros.

Primero leemos los datos y normalizamos:

In [1]:
from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step
training set (60000, 28, 28)
test set (10000, 28, 28)


## Partición training/validación

Nos quedaremos con un 80% de los datos para entrenar y un 20% para validar:

In [2]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

training set (48000, 784)
val set (12000, 784)


## Experimentación con conjunto de validación

Cualquier parámetro a modificar en nuestro modelo debería ser probado sobre el conjunto de validación y escoger aquel que produjera el mejor resultado para probar ese (y sólo ese) sobre el conjunto de test.

In [3]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD

batch_size=128
epochs=5

hdim=[128,256,512,1024]
best_acc=0.0
for h in hdim:
    model = Sequential()

    model.add(Input(784))
    model.add(Dense(h, activation='relu')) # <-- repetir esta línea tantas veces como número de capas ocultas se quiera
    model.add(Dense(num_classes, activation='softmax'))

    sgd=SGD(learning_rate=0.01, momentum=0.9)


    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



    history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val)) ## <--- OJO validation set

    if history.history['val_accuracy'][-1]>best_acc:
        best_acc=history.history['val_accuracy'][-1]
        besth=h

print("=============================")
print("Best acc",best_acc)
print("Best hidden dim",besth)
print("=============================")


## PROBAR EL MEJOR CON TEST
model = Sequential()

model.add(Input(784))
model.add(Dense(besth, activation='relu'))
model.add(Dense(num_classes, activation='softmax')) # la softmax no es una parametro como relu ya que estamos clasificando

sgd=SGD(learning_rate=0.01, momentum=0.9)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(x_train, y_train, # <--- aquí podríamos concatenar trainin+valid para no malgastar datos ////numpy concatenate
                    batch_size=batch_size, # el tamañi del batch influye mucho en gandes modelos de lenguaje
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))




Epoch 1/5
375/375 [==============================] - 6s 4ms/step - loss: 0.5370 - accuracy: 0.8508 - val_loss: 0.3094 - val_accuracy: 0.9147
Epoch 2/5
375/375 [==============================] - 1s 3ms/step - loss: 0.2818 - accuracy: 0.9195 - val_loss: 0.2561 - val_accuracy: 0.9283
Epoch 3/5
375/375 [==============================] - 1s 3ms/step - loss: 0.2329 - accuracy: 0.9345 - val_loss: 0.2179 - val_accuracy: 0.9388
Epoch 4/5
375/375 [==============================] - 1s 4ms/step - loss: 0.1999 - accuracy: 0.9437 - val_loss: 0.1962 - val_accuracy: 0.9443
Epoch 5/5
375/375 [==============================] - 2s 4ms/step - loss: 0.1765 - accuracy: 0.9501 - val_loss: 0.1784 - val_accuracy: 0.9490
Epoch 1/5
375/375 [==============================] - 2s 5ms/step - loss: 0.5247 - accuracy: 0.8567 - val_loss: 0.3055 - val_accuracy: 0.9137
Epoch 2/5
375/375 [==============================] - 1s 4ms/step - loss: 0.2716 - accuracy: 0.9233 - val_loss: 0.2384 - val_accuracy: 0.9338
Epoch 3/5
375

## EARLY STOPPING

De la multitud de parámetros a decidir en la definición y entrenamiento de las redes neuronales hay uno que podríamos automatizar. Es el número de epochs a emplear.** Dado que tenemos un conjunto de validación vamos a emplearlo para monitorizar cómo evoluciona la convergencia.** Si en algún momento el descenso por gradiente parece haber alcanzado una zona de meseta sobre dicho conjunto de validación podremos detener el entrenamiento.

In [4]:
model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)

callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) ## También podría ser 'loss' sin necesitar un conjunto de validación
                                                                              #patience si a lo largo de 3 epochs no varia dentro de ese 0.01 entonces para el entrenamiento, considera q no puede aprender mas
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

epochs=100  ## No nos preocupemos, el fit acabará antes por el early_stopping
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[callback])  # lista de funciones cada vez que acabe una epoch se ejecuta una funcion



Epoch 1/100
375/375 [==============================] - 2s 5ms/step - loss: 0.4936 - accuracy: 0.8676 - val_loss: 0.2916 - val_accuracy: 0.9178
Epoch 2/100
375/375 [==============================] - 1s 3ms/step - loss: 0.2628 - accuracy: 0.9269 - val_loss: 0.2350 - val_accuracy: 0.9340
Epoch 3/100
375/375 [==============================] - 1s 3ms/step - loss: 0.2134 - accuracy: 0.9410 - val_loss: 0.2023 - val_accuracy: 0.9437
Epoch 4/100
375/375 [==============================] - 1s 3ms/step - loss: 0.1803 - accuracy: 0.9494 - val_loss: 0.1734 - val_accuracy: 0.9526
Epoch 5/100
375/375 [==============================] - 1s 4ms/step - loss: 0.1552 - accuracy: 0.9570 - val_loss: 0.1569 - val_accuracy: 0.9567
Epoch 6/100
375/375 [==============================] - 2s 5ms/step - loss: 0.1361 - accuracy: 0.9623 - val_loss: 0.1421 - val_accuracy: 0.9602
Epoch 7/100
375/375 [==============================] - 1s 3ms/step - loss: 0.1215 - accuracy: 0.9668 - val_loss: 0.1354 - val_accuracy: 0.9628

## **Ejercicio**:

Entrenar una red neuronal MLP para el problema MNIST. Los valores que se quieren probar son los siguientes:

-   Número de capas ocultas [1,2,3]
-   Dimensión de las capas ocultas [512,1024]
-   Learning rate [0.01, 0.001, 0.0001]
-   Batch_size [128]
-   Epochs con early_stopping

Obtener la mejor combinación con el conjunto de validación y probarla finalmente sobre el conjunto de test

In [8]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD

batch_size=128
epochs=100

hdim=[512,1024]
best_acc=0.0
for h in hdim:
  for x in [1,2,3]:
      for l_rate in [0.01, 0.001, 0.0001]:
        model = Sequential()

        model.add(Input(784))

        for i in range(x) :
          model.add(Dense(h, activation='relu')) # <-- repetir esta línea tantas veces como número de capas ocultas se quiera
        model.add(Dense(num_classes, activation='softmax'))

        sgd=SGD(learning_rate=l_rate, momentum=0.9)
        callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

        model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])



        history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_val, y_val),
                        callbacks = [callback]) ## <--- OJO validation set

        if history.history['val_accuracy'][-1]>best_acc:
            best_acc=history.history['val_accuracy'][-1]
            besth=h
            bestx = x
            bestl = l_rate

print("=============================")
print("Best acc",best_acc)
print("Best number capa",bestx)
print("Best learning rate",bestl)
print("Best hidden dim",besth)
print("=============================")


## PROBAR EL MEJOR CON TEST
model = Sequential()

model.add(Input(784))
for i in range(bestx):
  model.add(Dense(besth, activation='relu'))
model.add(Dense(num_classes, activation='softmax')) # la softmax no es una parametro como relu ya que estamos clasificando

sgd=SGD(learning_rate=bestl, momentum=0.9)
callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(x_train, y_train, # <--- aquí podríamos concatenar trainin+valid para no malgastar datos ////numpy concatenate
                    batch_size=batch_size, # el tamañi del batch influye mucho en gandes modelos de lenguaje
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[callback])


Epoch 1/100
375/375 [==============================] - 2s 4ms/step - loss: 0.5086 - accuracy: 0.8595 - val_loss: 0.2986 - val_accuracy: 0.9169
Epoch 2/100
375/375 [==============================] - 1s 4ms/step - loss: 0.2673 - accuracy: 0.9248 - val_loss: 0.2395 - val_accuracy: 0.9342
Epoch 3/100
375/375 [==============================] - 2s 4ms/step - loss: 0.2165 - accuracy: 0.9399 - val_loss: 0.2009 - val_accuracy: 0.9442
Epoch 4/100
375/375 [==============================] - 1s 4ms/step - loss: 0.1834 - accuracy: 0.9491 - val_loss: 0.1765 - val_accuracy: 0.9498
Epoch 5/100
375/375 [==============================] - 1s 3ms/step - loss: 0.1590 - accuracy: 0.9561 - val_loss: 0.1589 - val_accuracy: 0.9558
Epoch 6/100
375/375 [==============================] - 1s 3ms/step - loss: 0.1402 - accuracy: 0.9618 - val_loss: 0.1461 - val_accuracy: 0.9585
Epoch 7/100
375/375 [==============================] - 1s 3ms/step - loss: 0.1256 - accuracy: 0.9658 - val_loss: 0.1349 - val_accuracy: 0.9620

## Model checkpoint

Los model checkpoint permiten almacenar el estado del modelo en un punto intermedio del entrenamiento. Esto es útil para poder recuperar el modelo en caso de que el entrenamiento se interrumpa por cualquier motivo. Así como para poder recuperar el mejor modelo obtenido durante el entrenamiento.


In [ ]:
batch_size=128
epochs=100

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

sgd=SGD(learning_rate=0.01, momentum=0.9)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])



callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[callback, checkpoint]) ## <--- dos callbacks, el early_stopping y el model_checkpoint


## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])





Ajustar parámetros
Es posible que en el ejemplo anterior el entrenamiento haya acabado prematuramente por el efecto del Early Stopping. En ese caso, el modelo que se ha obtenido no es el mejor posible. Para obtener el mejor modelo quizás sea necesario aumentar el número de epochs y para ello modificar algún parámetro del early stopping.

Ejercicio:
Modificar el early stopping y emplear un modelo con los mejores parámetros (número de capas ocultas, learning rate, batch size, etc) para evaluar finalmente el modelo guardado sobre el test.

In [ ]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD

batch_size=128
epochs=100

hdim=[512,1024]
best_acc=0.0
for h in hdim:
  for x in [1,2,3]:
      for l_rate in [0.01, 0.001, 0.0001]:
        model = Sequential()

        model.add(Input(784))

        for i in range(x) :
          model.add(Dense(h, activation='relu')) # <-- repetir esta línea tantas veces como número de capas ocultas se quiera
        model.add(Dense(num_classes, activation='softmax'))

        sgd=SGD(learning_rate=l_rate, momentum=0.9)
        callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)
        checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

        model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])



        history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_val, y_val),
                        callbacks = [callback,checkpoint]) ## <--- OJO validation set

        if history.history['val_accuracy'][-1]>best_acc:
            best_acc=history.history['val_accuracy'][-1]
            besth=h
            bestx = x
            bestl = l_rate

print("=============================")
print("Best acc",best_acc)
print("Best number capa",bestx)
print("Best learning rate",bestl)
print("Best hidden dim",besth)
print("=============================")


## PROBAR EL MEJOR CON TEST
model = Sequential()

model.add(Input(784))
for i in range(bestx):
  model.add(Dense(besth, activation='relu'))
model.add(Dense(num_classes, activation='softmax')) # la softmax no es una parametro como relu ya que estamos clasificando

sgd=SGD(learning_rate=bestl, momentum=0.9)
callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(x_train, y_train, # <--- aquí podríamos concatenar trainin+valid para no malgastar datos ////numpy concatenate
                    batch_size=batch_size, # el tamañi del batch influye mucho en gandes modelos de lenguaje
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[callback,checkpoint])
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
370/375 [============================>.] - ETA: 0s - loss: 0.5116 - accuracy: 0.8591
Epoch 1: val_accuracy improved from -inf to 0.91692, saving model to best_model.h5
375/375 [==============================] - 2s 4ms/step - loss: 0.5091 - accuracy: 0.8600 - val_loss: 0.2978 - val_accuracy: 0.9169
Epoch 2/100
366/375 [============================>.] - ETA: 0s - loss: 0.2685 - accuracy: 0.9245
Epoch 2: val_accuracy improved from 0.91692 to 0.93500, saving model to best_model.h5
375/375 [==============================] - 1s 4ms/step - loss: 0.2680 - accuracy: 0.9246 - val_loss: 0.2378 - val_accuracy: 0.9350
Epoch 3/100
368/375 [============================>.] - ETA: 0s - loss: 0.2175 - accuracy: 0.9392
Epoch 3: val_accuracy improved from 0.93500 to 0.94450, saving model to best_model.h5
375/375 [==============================] - 1s 3ms/step - loss: 0.2174 - accuracy: 0.9392 - val_loss: 0.2030 - val_accuracy: 0.9445
Epoch 4/100
367/375 [============================>.] - ETA: 0